In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Number of variables (adjust as needed for a test)
num_m = 5  # for i = 1, ..., m
num_k = 4  # for l = 0, ..., k-1

# Initialize random data for d, f, β, B, C, α
np.random.seed(42)  # for reproducibility
d = np.random.rand(num_m)       # Random demand for each i
f = np.random.rand(num_k)       # Random costs for each l
beta = np.random.rand(num_k, num_m)  # Random coefficients for constraints
B = 10.0                         # Capacity constraint
C = np.random.rand(num_m) * 5    # Upper bound constraints
alpha = np.random.rand()         # Random value for alpha

# Step 0: Initialization
u = np.zeros(num_k)  # Initial value for u
v = np.zeros(num_k)  # Initial value for v
z = np.zeros(num_m)  # Initial value for z

# Create a new model
model = gp.Model("algorithm_model")

# Define variables
y = model.addVars(num_m, vtype=GRB.BINARY, name="y")
t = model.addVars(num_k, vtype=GRB.CONTINUOUS, lb=0, name="t")
w = model.addVars(num_m, vtype=GRB.CONTINUOUS, lb=0, name="w")

# Step 1: Solve the master problem
# Objective function
objective = gp.quicksum(d[i] * y[i] for i in range(num_m)) + \
            gp.quicksum(f[l] * t[l] for l in range(num_k)) + alpha
model.setObjective(objective, GRB.MINIMIZE)

# Constraints
for l in range(num_k - 1):
    model.addConstr(
        gp.quicksum(z[i] * beta[l][i] for i in range(num_m)) + t[l] >= B,
        name=f"constraint1_{l}"
    )

for i in range(num_m):
    model.addConstr(w[i] <= C[i] * y[i], name=f"constraint2_{i}")
    model.addConstr(y[i] >= 0, name=f"nonnegative_y_{i}")

# Solve model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    y_values = model.getAttr("X", y)
    t_values = model.getAttr("X", t)
    w_values = model.getAttr("X", w)
    print("Optimal y values:", y_values)
    print("Optimal t values:", t_values)
    print("Optimal w values:", w_values)
    print("Objective Value:", model.objVal)

# Update objective with new values
L = sum(d[i] * y_values[i] + sum(f[l] * t_values[l] for l in range(num_k)) + alpha)
print("Updated Objective L:", L)

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 14 columns and 18 nonzeros
Model fingerprint: 0x5b9dc9ef
Variable types: 9 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 11.7681748
Presolve removed 13 rows and 14 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 11.7682 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.176817481587e+01, best bound 1.176817481587e+01, gap 0.0000%
Optimal y values: {0: 0.0, 1

TypeError: 'numpy.float64' object is not iterable